# First week

# Text preprocessing

## 1. Tokenization
Для начала рассматриваем текст как последовательность слов - токенов.\
Для этого необходимо разбивать текст на токены. Что и называется процессом токенизации.

## 2. Token normalization

1) Стемминг\
    Процесс избавления и замены суффиксов для получения корня слова, который называется стем\
    Используются эвристики для обрезания суффиксов.\
2) Лемматизация\
    Процесс словарного и/или морфологического анализ для приведения слова к "словарному" виду\
    Используются правила построения слов.

### 2.1 Пример: (nltk.stem.PorterStemmer)

Под капотом находится грамматика(та самая, которая из ТРЯПа), которая стемит слова.

In [4]:
import nltk

raw_words = ["feet", "wolves", "cats", "talked"]
stemmer = nltk.stem.PorterStemmer()

print("raw -> stemmed")

for word in raw_words:
    print(word + " -> " + stemmer.stem(word))

raw -> stemmed
feet -> feet
wolves -> wolv
cats -> cat
talked -> talk


### 2.2 Пример: (WordNet lemmatizer)
Используется база данных WordNet(WordNet — это лексическая база данных английского языка) для поиска лемм.

In [6]:
nltk.download('wordnet')
raw_words = ["feet", "wolves", "cats", "talked"]
stemmer = nltk.stem.WordNetLemmatizer()

print("raw -> lemmatized")

for word in raw_words:
    print(word + " -> " + stemmer.lemmatize(word))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Иван\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


raw -> lemmatized
feet -> foot
wolves -> wolf
cats -> cat
talked -> talked


# Feature extraction from text

## 1. Bag of words(BOW)

Идея состоит в том, чтобы посчитать появление каждого токена в нашем тексте.\
Мотивация: например в задаче сентимент-анализа, таким образом, будут помечаться слова с повышенной эмоциональной окраской, который могут стать сильным признаком для модели.\
Для каждого токена у нас есть столбец, такая техника называется: text vectorization.\
Минусы такого подхода:
- теряется порядок слов, поэтому и называется мешок слов\
- счетчики токенов не нормализованы

## 2. N-grams
Для того, чтобы учитывать какой-то порядок слов, мы можем рассматривать, пары, тройки слов и т.д.\
Минусы: 
- много фичей 


Данную проблему, можно решить, избавившись от некоторых n-грамм, основываясь на частоте их появления в текстах нашего корпуса.\
Нужно оставить лишь среднечастотные, т.к. высокочастотные н-граммы появляются почти во всех документах и ничего не значат, не способствуют разделению классов. В то же время, низкочастотные специфичны могут негативно повлиять на качество переобучением.

### 3. TF-IDF
#### TF - Term frequency
Частота появление токена или н-граммы t в документе d.\
tf(t,d) = $\frac{f_{t,d}}{\sum f_{t',d}}$, $\forall t' \in d$\
log-normalization: $1 + \log(f_{t,d})$

#### IDF - inverse document frequency
- $N = |D|$ - количество документов в корпусе
- $|\{d \in D: t\in d\}|$ - количество документо, в которых встречается токен t
- idf(t,d) = $\log \frac{N}{|\{d \in D: t\in d\}|}$

#### TF-IDF
tfidf(t, d, D) = tf(t, d) $\cdot$ idf(t, D)
Высокое значение tf-idf достигается за счет высокой частотности терма в документе и низкого количества документов, в котором это слово встречается.

### Пример:

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
texts = [
    "good movie", "not a good movie", "did not like", 
    "i like it", "good one"
]
# using default tokenizer in TfidfVectorizer
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
features = tfidf.fit_transform(texts)
pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names()
)

,good movie,like,movie,not
0,0.707107,0.000000,0.707107,0.000000
1,0.577350,0.000000,0.577350,0.577350
2,0.000000,0.707107,0.000000,0.707107
3,0.000000,1.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000


# Linear models for sentiment analysis


Логистическая регрессия + Bag of words является хорошим бейзлайном для работы с текстами.\
Глубокое обучение не дает сумасшедших добавок к качеству.

# Hashing trick for spam filtering

Если для обучения требуется очень много данных, то на хранение может потребоваться большое количество ресурсов, а если данные нужно параллельно синхронизировать, то совсем все плохо.\
В таком случае у нас существует HashingVectorizer, с помощью которого можно ограничивать количество итоговых фичей $\phi = \text{hash(ngramm)} \% 2^{b}$.\
В теории могут происходить колизии, которые будут ухудшать качество модели, но на практике, начиная с некоторого большого b, качество не отличается.\
Помимо этого существует следующая техника, которая учитывает персонализированные фичи для пользователя для спам-неспам классификации. А именно, добавить в модель фичи типа \<user_name>_\<n-gramm>.\
Данная техника дает существенное улучшение качества.

# Neural Networks for words
## 1. word2vec embeddings
Вместо разряженных матриц с onehotencoding векторами, в подходе с нейронными сетями используется более плотные представления слов и текста. Одним из таких подходов является word2vec embedding. Предобученная модель заменяет слово на плотный вектор с какими-то значениями, основное требование, чтобы слова с более похожими контекстами будут иметь почти коллиниарные вектора.\
Представлять текст будем суммированием таких векторов.(также как и для onehotencoding подхода).
## 2. 1D-Convolution
Лучшим решением будет представлять текст матрицей из последовательных векторов-слов, а поверх этой матрицы использовать сверточные фильтры.\
Подход заключается в том, чтобы взять наш текст, построить word2vec embedding'и, составить матрицу из этих строк-векоторов, далее прогнать 3,4,5 - gram скользящие окна со 100 разными фильтрами, на каждый фильтр мы будем использовать max pooling, что будет нам возвращать максимальное значение активации на каждом фильтре, получим фиксированный output вектор, который уже можно будет отправлять в модель. Что дает прирост в качестве.

# Neural networks for charachters
##  1. Текст как последовательность букв
В отличие от предыдущих подходов, здесь мы рассмотрим текст как последовательность символов.\
Допустим у нас есть алфавит символов. Сделаем ohe для каждого символа и запустим сверточные фильтры поверх них и используем скользящее окно max pooling для двух соседних значений для каждого слоя, потом получим матрицу, поверх нее снова повторим процесс.\
По оценкам качества на больших датасетах, подходы глубокого обучения выигрывают у классических методов.